 <font size=6> **实验15 循环神经网络**</font>

* 了解序列数据的特点，理解自回归模型、隐变量自回归模型等统计工具的基本用法 ；
* 理解文本预处理的内容和实现方法，理解语言模型中的n 元语法和长序列数据读取方法；
* 理解隐状态、循环神经网络；
* 理解RNN的从零开始实现、简洁实现，并能加以应用。


**(实验报告提交题)**  lab_15_4目录中spider.ipynb程序是根据https://zhuanlan.zhihu.com/p/597957245 站点代码改编的(原url无效)，其功能是爬取李白诗词并保存到libai.txt.文件中，请据此和RNN实现李白诗词生成的功能。

**结果参考：**


# 导入库

In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
import numpy as np
import random

# 数据加载和处理

## 加载数据

In [3]:
# 数据处理
with open('libai.txt', 'r', encoding='utf8') as file:
    content = file.read().replace('\n', '')
# print(content.replace('\n', ''))
chars = list(set(content))
print(chars)
n_chars = len(chars)
print(n_chars)
# 字符与对应数字标记
char_indices = dict((c, i) for i, c in enumerate(chars))
print(char_indices)
# 数字标记与对应的字符
indices_char = dict((i, c) for i, c in enumerate(chars))
print(indices_char)

maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(content) - maxlen, step):
    sentences.append(content[i: i+maxlen])
    next_chars.append(content[i+maxlen])

['征', '进', '管', '涛', '帆', '略', '饯', '富', '用', '班', '还', '侠', '挥', '暗', '谈', '屏', '提', '旄', '恨', '栈', '涂', '蜀', '娥', '蒲', '固', '部', '存', '居', '布', '迟', '幽', '毅', '兀', '天', '休', '屐', '飘', '匹', '冲', '乌', '途', '涧', '心', '经', '庐', '逻', '聚', '蔓', '亲', '铁', '空', '眉', '州', '银', '净', '南', '雁', '雳', '吹', '愚', '乃', '定', '震', '瑶', '盈', '垓', '柳', '带', '寂', '巅', '渠', '樽', '窥', '底', '呼', '夜', '箭', '翁', '入', '绕', '覆', '掷', '殷', '夏', '引', '桐', '任', '是', '(', '甲', '似', '度', '嚱', '钓', '达', '尝', '蔡', '狂', '夙', '随', '鹏', '忘', '伤', '餐', '试', '貂', '得', '缦', '促', '满', '静', '惊', '号', '虚', '苍', '歌', '湍', '放', '珑', '剡', '流', '鸪', '吮', '竹', '凄', '北', '牙', '百', '渌', '仙', '才', '丘', '宾', '朓', '通', '峥', '晨', '许', '机', '殊', '讵', '池', '汪', '总', '至', '肝', '守', '红', '挟', '夕', '垂', '思', '惑', '惧', '郭', '隔', '贤', '楼', '孟', '桓', '眼', '际', '犬', '坠', '及', '哀', '吸', '眠', '昭', '依', '赊', '练', '条', '峦', '闺', '岭', '姑', '言', '怜', '之', '我', '帏', '肠', '春', '悲', '跪', '醉', '奴', '转', '处', '鸟', '数', '歇', '投', '醒', '崖', '下', '遇', '吠', '鸾',

In [4]:
char_indices

{'征': 0,
 '进': 1,
 '管': 2,
 '涛': 3,
 '帆': 4,
 '略': 5,
 '饯': 6,
 '富': 7,
 '用': 8,
 '班': 9,
 '还': 10,
 '侠': 11,
 '挥': 12,
 '暗': 13,
 '谈': 14,
 '屏': 15,
 '提': 16,
 '旄': 17,
 '恨': 18,
 '栈': 19,
 '涂': 20,
 '蜀': 21,
 '娥': 22,
 '蒲': 23,
 '固': 24,
 '部': 25,
 '存': 26,
 '居': 27,
 '布': 28,
 '迟': 29,
 '幽': 30,
 '毅': 31,
 '兀': 32,
 '天': 33,
 '休': 34,
 '屐': 35,
 '飘': 36,
 '匹': 37,
 '冲': 38,
 '乌': 39,
 '途': 40,
 '涧': 41,
 '心': 42,
 '经': 43,
 '庐': 44,
 '逻': 45,
 '聚': 46,
 '蔓': 47,
 '亲': 48,
 '铁': 49,
 '空': 50,
 '眉': 51,
 '州': 52,
 '银': 53,
 '净': 54,
 '南': 55,
 '雁': 56,
 '雳': 57,
 '吹': 58,
 '愚': 59,
 '乃': 60,
 '定': 61,
 '震': 62,
 '瑶': 63,
 '盈': 64,
 '垓': 65,
 '柳': 66,
 '带': 67,
 '寂': 68,
 '巅': 69,
 '渠': 70,
 '樽': 71,
 '窥': 72,
 '底': 73,
 '呼': 74,
 '夜': 75,
 '箭': 76,
 '翁': 77,
 '入': 78,
 '绕': 79,
 '覆': 80,
 '掷': 81,
 '殷': 82,
 '夏': 83,
 '引': 84,
 '桐': 85,
 '任': 86,
 '是': 87,
 '(': 88,
 '甲': 89,
 '似': 90,
 '度': 91,
 '嚱': 92,
 '钓': 93,
 '达': 94,
 '尝': 95,
 '蔡': 96,
 '狂': 97,
 '夙': 98,
 '随': 99,
 '鹏': 100,

In [5]:
n_chars

1472

In [6]:
sentences[:10]

['将进酒君不见黄河之水天上来，奔流到海不复',
 '君不见黄河之水天上来，奔流到海不复回。君',
 '黄河之水天上来，奔流到海不复回。君不见高',
 '水天上来，奔流到海不复回。君不见高堂明镜',
 '来，奔流到海不复回。君不见高堂明镜悲白发',
 '流到海不复回。君不见高堂明镜悲白发，朝如',
 '不复回。君不见高堂明镜悲白发，朝如青丝暮',
 '。君不见高堂明镜悲白发，朝如青丝暮成雪。',
 '见高堂明镜悲白发，朝如青丝暮成雪。人生得',
 '明镜悲白发，朝如青丝暮成雪。人生得意须尽']

In [7]:
next_chars[:10]

['回', '不', '堂', '悲', '，', '青', '成', '人', '意', '欢']

## one-hot编码

In [8]:
# 将所有句子中的字符转换为独热编码的形式
import numpy as np
import torch
X_train = torch.zeros((len(sentences), maxlen, len(chars)), dtype=torch.float32)
y_train = torch.zeros((len(sentences), len(chars)), dtype=torch.float32)

for i, sentence in enumerate(sentences):
    for j, char in enumerate(sentence):
        X_train[i, j, char_indices[char]] = 1
    y_train[i, char_indices[next_chars[i]]] = 1

In [9]:
X_train[0][0]

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [10]:
len(X_train[0][0])

1472

# RNN建模

In [11]:
class RNNModel(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
      super().__init__()
      self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
      self.linear = nn.Linear(hidden_size, output_size)
      self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x):
      out,_ = self.rnn(x)
      out = out[:, -1, :]
      return self.fc(out)


model = RNNModel(n_chars, 128, n_chars)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(X_train.float())
    # y_train needs to be long type for CrossEntropyLoss
    loss = criterion(output, y_train.argmax(dim=1))
    loss.backward()
    optimizer.step()
    print(f'Epoch: [{epoch+1}], Loss: {loss.item():.4f}')

Epoch: [1], Loss: 7.3081
Epoch: [2], Loss: 7.1123
Epoch: [3], Loss: 6.4363
Epoch: [4], Loss: 6.1708
Epoch: [5], Loss: 6.0408
Epoch: [6], Loss: 5.9916
Epoch: [7], Loss: 5.9532
Epoch: [8], Loss: 5.9247
Epoch: [9], Loss: 5.8831
Epoch: [10], Loss: 5.8358
Epoch: [11], Loss: 5.8603
Epoch: [12], Loss: 5.8103
Epoch: [13], Loss: 5.7894
Epoch: [14], Loss: 5.7451
Epoch: [15], Loss: 5.7024
Epoch: [16], Loss: 5.6514
Epoch: [17], Loss: 5.5960
Epoch: [18], Loss: 5.5284
Epoch: [19], Loss: 5.4571
Epoch: [20], Loss: 5.3799
Epoch: [21], Loss: 5.2877
Epoch: [22], Loss: 5.1783
Epoch: [23], Loss: 5.1019
Epoch: [24], Loss: 5.0242
Epoch: [25], Loss: 4.9133
Epoch: [26], Loss: 4.8238
Epoch: [27], Loss: 4.7090
Epoch: [28], Loss: 4.5873
Epoch: [29], Loss: 4.4444
Epoch: [30], Loss: 4.2594
Epoch: [31], Loss: 4.1062
Epoch: [32], Loss: 3.9604
Epoch: [33], Loss: 3.7959
Epoch: [34], Loss: 3.6470
Epoch: [35], Loss: 3.4851
Epoch: [36], Loss: 3.3182
Epoch: [37], Loss: 3.1506
Epoch: [38], Loss: 2.9838
Epoch: [39], Loss: 2.

In [17]:
def sample(preds, temperature=1.0):
    preds = torch.softmax(preds / temperature, dim=-1)
    preds = preds.detach().cpu().numpy()
    return np.random.choice(len(preds), p = preds)

def generate_text(length, diversity):
  start_index = random.randint(0, len(content) - maxlen - 1)
  sentence = content[start_index: start_index + maxlen]
  generate = ''
  for i in range(length):
    x_pred = torch.zeros((1, maxlen, len(chars)), dtype=torch.float32)
    for t, char in enumerate(sentence):
      x_pred[0, t, char_indices[char]] = 1
    preds = model(x_pred.float())[0]
    next_index = sample(preds, diversity)
    next_char = indices_char[next_index]
    sentence = sentence[1:] + next_char
    generate += next_char
  return generate

In [18]:
generate_text(24, 0.9)

'樽天雪长安。八后蝴蝶笑，一作：连欲)，五娥亦。春'